In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
data_pars = pd.read_excel('data/Data_TSUM.xlsx',sheet_name='Data_Parsing')
data_comp = pd.read_excel('data/Data_TSUM.xlsx',sheet_name='Data_Company')
display(data_pars.head())
display(data_comp.head())



,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188
2,Valentino,Shoes,aaaaa1111_13,black,184
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250


,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153


In [88]:
pars_copy = data_pars.copy()
pars_copy['producer_id']= pars_copy['producer_id'].replace(
    to_replace={r'_':'',r'-':'',r'~':'',r'\\':'',r'//':'',r'/':''}, regex=True
    )
pars_copy['producer_color']= pars_copy['producer_color'].replace(
    to_replace={r'_':'',r'-':'',r'~':'',r'\\':'',r'//':'',r'/':''}, regex=True
    )
pars_copy = pars_copy.merge(data_comp.loc[:,['item_id','color_id','current price']],how='left',right_on=['item_id', 'color_id'],left_on=['producer_id','producer_color'])

pars_copy['percent'] = pars_copy['price']/pars_copy['current price']
display(pars_copy)

,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,percent
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,0.676113
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,1.167702
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,0.786325
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,1.173653
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,1.633987
...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165,0.993939
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196,0.806122
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236,0.822034
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222,1.153153


In [90]:
pars_copy['Brend-cat'] = pars_copy['brand']+' / '+pars_copy['Category']
display(pars_copy)

,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,percent,Brend-cat
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,0.676113,Valentino / Shoes
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,1.167702,Valentino / Shoes
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,0.786325,Valentino / Shoes
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,1.173653,Valentino / Shoes
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,1.633987,Valentino / Shoes
...,...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165,0.993939,Stone Island / Bags
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196,0.806122,Stone Island / Bags
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236,0.822034,Stone Island / Bags
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222,1.153153,Stone Island / Bags


In [98]:
fig = px.scatter(
    data_frame=pars_copy,
    x='percent',
    y='Brend-cat',
    color_discrete_sequence=px.colors.qualitative.Set3,
    )
fig.show()

In [97]:
def outliers_iqr(data, feature,log_scale=False,left=1.5,right=1.5):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return cleaned
display(pars_copy['Brend-cat'].unique())
pars_copy2 = pars_copy.drop(labels=range(0,len(pars_copy)))
for name in pars_copy['Brend-cat'].unique():
    pars_copy2 = pd.concat([pars_copy2,outliers_iqr(pars_copy[pars_copy['Brend-cat'] == name], 'percent',left=0.7,right=0.7)])

#print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
#print(f'Результирующее число записей: {cleaned.shape[0]}')
fig = px.scatter(
    data_frame=pars_copy2,
    x='percent',
    y='Brend-cat',
    color_discrete_sequence=px.colors.qualitative.Set3,
    )
fig.show()
pars_copy2.info()

array(['Valentino / Shoes', 'Valentino / Bags', 'Brioni / Shoes',
       'Brioni / Bags', 'Stone Island / Shoes', 'Stone Island / Bags'],
      dtype=object)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 0 to 74
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           63 non-null     object 
 1   Category        63 non-null     object 
 2   producer_id     63 non-null     object 
 3   producer_color  63 non-null     object 
 4   price           63 non-null     int64  
 5   item_id         63 non-null     object 
 6   color_id        63 non-null     object 
 7   current price   63 non-null     int64  
 8   percent         63 non-null     float64
 9   Brend-cat       63 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 5.4+ KB
